In [ ]:
# -*- coding=utf-8 -*-
from queue import Queue
from threading import Thread
import threading
import time

from lxml import etree
import requests

import re
url_list = []
actor_list = []
record_movie = []
lock = threading.Lock()
TotalMovie = 15000

class YahooSpider(Thread):
    def __init__(self, q) :
        # 重寫寫父類的__init__方法
        super(YahooSpider, self).__init__()
        self.q = q
        
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        }

    def run(self):
        self.parse_movie()

    def send_request(self, url):
        '''
        用來傳送請求的方法
        :return: 返回網頁原始碼
        '''
        # 請求出錯時，重複請求３次,
        i = 0
        while i <= 3:
            try:
                print
                u"[INFO]請求url:" + url
                html = requests.get(url=url, headers=self.headers).content
                
            except Exception as e:
                print
                u'[INFO] %s%s' % (e, url)
                i += 1
            else:
                return html
#44行，沒超過50...
    def parse_movie(self):
        index = 1
        with open('Movie.csv', 'a') as movieFile:
            movieFile.truncate(0)
            while len(record_movie) <= TotalMovie:
                try:
                    url = url_list.pop()
                except IndexError as e:
                    break
                if url in record_movie:
                    continue
                else:
                    record_movie.append(url)
                    list(set(record_movie))
                    lock.acquire()
                    response = self.send_request(url)
                    html = etree.HTML(response)
                    # 上映日期
                    date = html.xpath('//div[@class="movie_intro_info_r"]/span/text()')[0]
                    # 電影名稱
                    title = html.xpath('//div[@class="movie_intro_info_r"]/h1/text()')[0]
                    # 分類
                    level_name = html.xpath('//div[@class="level_name"]/a/text()')
                    levels = ''
                    for level in level_name:
                        levels += (str(level) + " ")
                    # 劇情介紹
                    storyText = html.xpath('//span[@id="story"]/text()')
                    Str = ''
                    for story in storyText:
                        Str += story
                    #演員連結
                    ActorHtml = html.xpath('//a/@href')
                    for Actor in ActorHtml:
                        if 'name_main' in Actor and Actor not in actor_list:
                            actor_list.append(Actor)
                    # 將每一部電影的名稱跟評分加入到佇列
                    AddText = ''
                    AddText = str(title) + " | " + str(levels) + "| " +str(date) + " | " + str(Str.replace("\n", ""))
                    movieFile.write(AddText+"\n")
                    while 1:
                        urlAct = actor_list.pop()
                        r = requests.get(urlAct, headers=self.headers)
                        r.encoding = 'utf-8'
                        htmlAct = etree.HTML(r.text)
                        # 電影html
                        MovieHtml = htmlAct.xpath('//a/@href')
                        for Movie in MovieHtml:
                            if 'movieinfo_main' in Movie and Movie not in url_list:
                                url_list.append(Movie)
                    index += 1
                    lock.release()
                    
class InitUrlSpider(Thread):
    def run(self):
        url = 'https://movies.yahoo.com.tw/movieinfo_main/%E9%BB%91%E5%AF%A1%E5%A9%A6-black-widow-10397' 
        #url = 'https://movies.yahoo.com.tw/movieinfo_main/%E6%BF%80%E6%83%85%E6%97%A5%E6%9C%AC-%E4%BB%8A%E6%9D%91%E6%98%8C%E5%B9%B3%E7%9A%84%E6%98%A0%E7%95%AB%E6%B5%AE%E7%94%9F%E9%8C%84-deadly-passion-japanthe-films-of-shohei-imamura-6312'
        url_list.append(url)
        
        # 構造所有動畫類ｕｒｌ
        base_url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page='
        base_url2 = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page='
        base_url3 = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=16&page='
        for num in range(1, 46):
            actor_list.append(base_url + str(num))
            actor_list.append(base_url2 + str(num))
            actor_list.append(base_url3 + str(num))
            

def main():
    # 建立一個佇列用來儲存程式獲取到的資料
    qData = Queue()
    a = InitUrlSpider()
    a.start()

    # 儲存執行緒
    Thread_list = []
    # 建立並啟動執行緒
    for i in range(8):
        p = YahooSpider(qData)
        p.start()
        Thread_list.append(p)

    a.join()
    # 讓主執行緒等待子執行緒執行完成
    for i in Thread_list:
        i.join()
        
if __name__ == "__main__":
    start = time.time()
    main()
    print('[info]耗時：%s' % (time.time() - start))
    